In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
from typing import List
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import xgboost

import networkx
from networkx.algorithms.traversal.depth_first_search import dfs_edges

import warnings
warnings.filterwarnings('ignore')

/var/folders/6y/6p7qrh9s27d1s3l8p2l81vbw0000gn/T/ipykernel_70727/4219629903.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [2]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
data = dict()
for dirname, _, filenames in os.walk('../kaggle/input'):
    for filename in filenames:
        table_name = filename.split('.')[0]
        table_path = os.path.join(dirname, filename)
        try:
            data[table_name] = pd.read_csv(table_path)
        except UnicodeDecodeError:
            data[table_name] = pd.read_csv(table_path, encoding='cp1252')
        except Exception as e:
            print(f"Error with {filename}: {e}")

In [3]:
# Split dict of dataframes by gender and other (supplemental) data
mens_data = dict()
womens_data = dict()
supplemental_data = dict()

for k, v in data.items():
    if k.startswith("M"):
        mens_data[k] = v
    elif k.startswith("W"):
        womens_data[k] = v
    else:
        supplemental_data[k] = v
        

In [70]:
kenpom = pd.read_csv("../data/kenpom/kenpom_filtered.csv")

In [71]:
kenpom

,TeamID,TeamName,Season,FG2Pct,FG3Pct,FTPct,OppFG2Pct,OppFG3Pct,StlRate,OppStlRate,FTRate_offense,FTRate_defense,TOPct_offense,ORPct_offense,HgtEff,Bench,AdjTempo,AdjOE,AdjDE
0,1102,Air Force,2007,56.552500,39.765300,75.742600,49.192800,32.328300,0.107500,0.076300,37.546500,28.787900,16.652300,24.440900,-0.75,15.21,56.6656,116.2860,97.0562
1,1103,Akron,2007,52.701600,39.285700,67.244100,43.041000,33.667300,0.118800,0.083500,35.455100,34.328400,18.560600,36.816900,-1.05,28.18,63.3180,115.1090,98.9272
2,1104,Alabama,2007,49.416300,37.122600,70.764600,48.145200,30.822900,0.083300,0.107700,36.093100,24.899200,20.110600,38.921100,0.78,28.57,64.8798,113.9920,101.5170
3,1105,Alabama A&M,2007,43.212400,26.226000,63.475200,43.026400,31.368400,0.093600,0.131100,39.330500,39.821900,24.339700,29.131500,0.58,33.08,70.3008,83.7228,106.7230
4,1106,Alabama St,2007,39.767200,30.178600,64.067300,49.504100,29.438200,0.092700,0.093200,41.106200,42.471000,20.809300,33.832200,-1.42,26.83,63.4575,90.8625,108.7510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4625,1460,Wright St,2024,58.800623,38.418079,76.026273,55.042735,37.146893,0.084446,0.102607,33.553719,30.031949,16.526018,29.113924,-0.13,19.77,71.4877,116.1700,115.4110
4626,1461,Wyoming,2024,49.036609,36.065574,74.920128,53.660797,34.401114,0.073769,0.115783,37.985437,27.824151,18.515578,28.686058,1.18,24.34,67.1859,108.9400,107.9180
4627,1462,Xavier,2024,47.997139,34.042553,74.429224,49.349656,33.378933,0.098011,0.086555,31.955253,30.029441,16.165388,32.081633,0.37,29.68,71.0335,113.0130,99.5332
4628,1463,Yale,2024,52.788462,34.744268,71.788991,48.947951,34.466019,0.088632,0.081431,27.131301,29.388560,14.125761,28.384279,0.54,25.91,64.7527,112.0220,102.7700


In [72]:
def get_season_stats(dataset, detailed=False, post_season=False, year=2024):
    # Gets the first letter in dataset
    gender = list(dataset.keys())[0][0]
    
    if detailed:
        if post_season:
            df = dataset[f"{gender}NCAATourneyDetailedResults"]
        else:
            df = dataset[f"{gender}RegularSeasonDetailedResults"]
        
    else:
        if post_season:
            df = dataset[f"{gender}NCAATourneyCompactResults"]
        else:
            df = dataset[f"{gender}RegularSeasonCompactResults"]
        
    df = df[df["Season"] == year]
    return df, gender

def compute_margins_of_victory(df):
    df["margin"] = df["WScore"] - df["LScore"]
    
    win_df = df[["WTeamID", "margin"]].rename(columns={"WTeamID": "TeamID"})
    lose_df = df[["LTeamID", "margin"]].rename(columns={"LTeamID": "TeamID"})
    lose_df["margin"] = -lose_df["margin"]

    res = pd.concat([win_df, lose_df], axis=0)
    return res.groupby("TeamID")["margin"].mean()

def join_team_names(df, data, gender="M"):
    """
    df: pd.DataFrame
        dataframe appending teams to
    data: dict[str, pd.DataFrame]
        dictionary of all table names and data
    """
    res = pd.merge(df, data[f"{gender}Teams"][["TeamID", "TeamName"]], on="TeamID")
    return res

def create_srs(df,gender):

    df["margin"] = df["WScore"] - df["LScore"]
    win_df = df[["WTeamID", "margin", "LTeamID"]].rename(
        columns={"WTeamID": "team_id", "LTeamID": "opp_id"}
    )
    lose_df = df[["WTeamID", "margin", "LTeamID"]].rename(
        columns={"LTeamID": "team_id", "WTeamID": "opp_id"}
    )
    lose_df["margin"] = -lose_df["margin"]

    teams = pd.concat([win_df, lose_df], axis=0)
    spreads = compute_margins_of_victory(df)
    
    terms = []
    solutions = []

    for team_id in spreads.keys():
        row = []
        opps = list(teams[teams["team_id"] == team_id]["opp_id"])

        for opp_id in spreads.keys():
            if opp_id == team_id:
                # coef for the team itself should be 1
                row.append(1)
            elif opp_id in opps:
                # coef for opponents is 1 over num of opps
                row.append(-1.0/len(opps))
            else:
                # teams not faced get a 0 coef
                row.append(0)
        terms.append(row)

        solutions.append(spreads[team_id])

    solutions, _, _, _ = np.linalg.lstsq(np.array(terms), np.array(solutions), rcond=None)
    
    ratings = list(zip( spreads.keys(), solutions ))
    srs = pd.DataFrame(ratings, columns=['team', 'rating'])
    rankings = srs.sort_values('rating', ascending=False).reset_index()[['team', 'rating']]
    rankings = join_team_names(rankings.rename(columns={"team": "TeamID"}), data, gender=gender)
    return rankings

def get_coach_win_perc(
    dataset: dict,
    regular_season: bool,
    year:int = 2024
) -> pd.DataFrame:
    """
    
    parameters
    ----------
    dataset: dict
        dictionary of datasets to use. it will be
        mens_data or womens_data.
        
    year: int
        year to filter data. it will get coaches stats for everything
        up until this year. (model can't have any look ahead bias). for post
        season games, use a year one less than the year of interest.
        
    returns
    -------
    coaches_stats: pd.DataFrame
        dataframe with count of wins, win percentage, and std dev
        of wins.
    """
    
    # Gets the first letter in dataset
    gender = list(dataset.keys())[0][0]
    
    if regular_season:
        df = dataset[f"{gender}RegularSeasonCompactResults"]
        #Filter season up until season of interest
        df = df[df["Season"] <= year]
    else:
        df = dataset[f"{gender}NCAATourneyCompactResults"]
        #Filter season up until season of interest
        df = df[df["Season"] < year]
        
    
    
    winning_coaches_df = pd.merge(
        df,
        dataset[f"{gender}TeamCoaches"],
        how="left",
        left_on=["Season", "WTeamID"],
        right_on=["Season", "TeamID"]
    )

    winning_coaches_df = winning_coaches_df[
        (winning_coaches_df['DayNum'] >= winning_coaches_df['FirstDayNum']) 
        & (winning_coaches_df['DayNum'] <= winning_coaches_df['LastDayNum'])
    ]
    winning_coaches_df["win"] = 1

    #Make sure the join dind't create dupes
    assert len(winning_coaches_df) == len(df)

    losing_coaches_df = pd.merge(
        df,
        dataset[f"{gender}TeamCoaches"],
        how="left",
        left_on=["Season", "LTeamID"],
        right_on=["Season", "TeamID"]
    )

    losing_coaches_df = losing_coaches_df[
        (losing_coaches_df['DayNum'] >= losing_coaches_df['FirstDayNum']) 
        & (losing_coaches_df['DayNum'] <= losing_coaches_df['LastDayNum'])
    ]
    losing_coaches_df["win"] = 0

    #Make sure the join dind't create dupes
    assert len(losing_coaches_df) == len(df)

    coaches_df = pd.concat(
        [
            losing_coaches_df[["CoachName", "win"]],
            winning_coaches_df[["CoachName", "win"]]
        ],
        axis=0
    )

    coach_stats = (
        coaches_df
        .groupby("CoachName")["win"]
        .describe()
        .sort_values("count", ascending=False)
        [["count", "mean", "std"]]
        .fillna(0)
    )

    return coach_stats

def get_system_ratings(
    mens_dataset, #There are only ratings for men
    systems: List[str],
    year: int=2024,
):
    """
    gets system ratings for each team for specified systems for a specific year.
    
    parameters
    ---------
    mens_dataset: dict
        dictionary of datasets for men
    systems: List[str]
        list of dictionaries we are interested in seeing
    year: int
        year to look for ratings
    moving_average: str
        specifies how to calculate rolling ratings for given systems.
        if None, the system takes the most recent system rating
    
    returns
    -------
    df: pd.DataFrame
        data that reflects ratings for a team
    """
    
    # Filter by season - only take most recent
    df = mens_dataset["MMasseyOrdinals"]
    df = df[df["Season"] == year]
    
    # Filter by system
    df = df[df["SystemName"].isin(systems)]
    
    latest_rank = (
        df
        .sort_values("RankingDayNum")
        .groupby(["TeamID","SystemName"])
        ["OrdinalRank"]
        .last()
        .unstack("SystemName")
        .reset_index().
        rename(columns = {i: i+"_latest" for i in systems})
    )
    
    transformed_df = (
        df
        .sort_values(by="RankingDayNum")
        .groupby(["TeamID", "SystemName"], group_keys=False)
        ["OrdinalRank"]
        .rolling(5) # TODO: Parameterize this (window and moving average method)
        .mean()
        .unstack("SystemName")
        .reset_index()
        .drop("level_1", axis=1)
        .groupby("TeamID")
        [systems]
        .last()
        .reset_index()
        .rename(columns = {i: i+"_rolling" for i in systems})
    )
    
    res = pd.merge(latest_rank, transformed_df, on="TeamID")

    return res

def get_post_season(data, year):
    
    df, gender = get_season_stats(
            data, 
            detailed=False, 
            post_season=True, 
            year=year
    )
    
    # Shuffle teams for positional encoding (model shouldn't have winning teams features as the same)
    df["TeamID"] = np.where(
        np.random.uniform(0,1, size=len(df)) > .5, 
        df["WTeamID"], 
        df["LTeamID"]
    )
    df["team_score"] = np.where(
        df["TeamID"] == df["WTeamID"], 
        df["WScore"], 
        df["LScore"]
    )
    df["OppID"] = np.where(
        df["TeamID"] == df["WTeamID"], 
        df["LTeamID"], 
        df["WTeamID"]
    )
    df["opp_score"] = np.where(
        df["TeamID"] == df["WTeamID"], 
        df["LScore"], 
        df["WScore"]
    )
    df = df.drop(
        ["WTeamID", "LTeamID", "WScore", "LScore", "WLoc", "NumOT"],
        axis=1
    )
    
    return df

def get_features(mens_data, year, systems):
    # Season Stats
    df, gender = get_season_stats(
        mens_data, 
        detailed=False, 
        post_season=False, 
        year=year
    )

    # Rating System
    srs = create_srs(df, gender)

    # System Ratings
    system_ratings = get_system_ratings(
        mens_data, 
        systems=systems
    ) #KenPom, Nolan ELO, EPSN BPI

    # Ratings df
    ratings_df = pd.merge(
                srs,
                system_ratings,
                on="TeamID"
    )

    # Coaches postseason win stats
    coaches_postseason_win_df = get_coach_win_perc(
        dataset=mens_data, 
        regular_season=False, 
        year=year
    ).rename(columns={"count": "count_post", "mean": "mean_post", "std": "std_post"})

    # Coaches regular season win stats
    coaches_regseason_win_df = get_coach_win_perc(
        dataset=mens_data, 
        regular_season=True, 
        year=year
    ).rename(columns={"count": "count_reg", "mean": "mean_reg", "std": "std_reg"})

    coaches_df = pd.merge(
        coaches_regseason_win_df,
        coaches_postseason_win_df,
        on="CoachName",
        how="left"
    ).fillna(0)

    # Get coaches for the year and only grab the most recent coach for a certain team
    curr_coaches = (
        mens_data["MTeamCoaches"][
            mens_data["MTeamCoaches"]["Season"] == year
        ]
        .sort_values("FirstDayNum")
        .groupby("TeamID")["CoachName"]
        .last()
        .reset_index()
    )

    # Get coach stats for current coaches
    coaches_df = pd.merge(
        curr_coaches,
        coaches_df,
        on="CoachName",
        how="left"
    )


    feature_df = (
        pd.merge(
            ratings_df,
            coaches_df
        )
        .drop(["TeamName", "CoachName"], axis=1)
    )

    feature_df = pd.merge(
        feature_df,
        kenpom[kenpom["Season"] == year],
        on="TeamID",
        how="left"
    )
    
    return feature_df


def merge_features_to_games(feature_df, post_season_df, year, training=True):
    
    post_season_merged = pd.merge(
        pd.merge(
            feature_df,
            post_season_df,
            on="TeamID",
        ),
        feature_df,
        left_on="OppID",
        right_on="TeamID",
        suffixes=("_team", "_opp")
    )
    if training:
        post_season_merged["win"] = post_season_merged["team_score"] > post_season_merged["opp_score"]
        post_season_merged = (
            post_season_merged
            .drop(
                ["TeamID_team", "team_score", "OppID", "TeamID_opp", "opp_score", "DayNum"], 
                axis=1
            )
        )
    return post_season_merged

# This function will change a lot based on what we are trying to predict
# Simplest training method is to grab team ids from previous years and pull in reg season stats to make a prediction
# what we should try to get to is running simulations and making predictions based on matchups then have some sort of loss metric for how good or bad a bracket is.
# Also adding stats like if they're on a run or not would be cool (tough to do at inference time)
def create_mens_training_data():
    
    training_data = dict()
    
    for year in tqdm(range(2003, 2024)):
        
        feature_df = get_features(mens_data, year=year, systems=["POM", "NOL", "EBP"])
        post_season_df = get_post_season(mens_data, year)
        post_season_merged = merge_features_to_games(feature_df, post_season_df, year)
        
        training_data[year] = post_season_merged
    
    return training_data

In [73]:
train_dict = create_mens_training_data()

100%|██████████| 21/21 [00:56<00:00,  2.70s/it]


In [74]:
all_data = pd.concat(train_dict.values(), ignore_index=True)

In [75]:
all_data.groupby("Season").count()

,rating_team,EBP_latest_team,NOL_latest_team,POM_latest_team,POM_rolling_team,NOL_rolling_team,EBP_rolling_team,count_reg_team,mean_reg_team,std_reg_team,...,FTRate_offense_opp,FTRate_defense_opp,TOPct_offense_opp,ORPct_offense_opp,HgtEff_opp,Bench_opp,AdjTempo_opp,AdjOE_opp,AdjDE_opp,win
Season,,,,,,,,,,,,,,,,,,,,,
2007.0,61,61,61,61,61,61,61,61,61,61,...,61,61,61,61,61,61,61,61,61,61
2008.0,59,59,59,59,59,59,59,59,59,59,...,59,59,59,59,59,59,59,59,59,59
2009.0,61,61,61,61,61,61,61,61,61,61,...,61,61,61,61,61,61,61,61,61,61
2010.0,62,62,62,62,62,62,62,62,62,62,...,62,62,62,62,62,62,62,62,62,62
2011.0,64,64,64,64,64,64,64,64,64,64,...,64,64,64,64,64,64,64,64,64,64
2012.0,61,61,61,61,61,61,61,61,61,61,...,61,61,61,61,61,61,61,61,61,61
2013.0,57,57,57,57,57,57,57,57,57,57,...,57,57,57,57,57,57,57,57,57,57
2014.0,55,55,55,55,55,55,55,55,55,55,...,55,55,55,55,55,55,55,55,55,55
2015.0,63,63,63,63,63,63,63,63,63,63,...,63,63,63,63,63,63,63,63,63,63


In [36]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(all_data[all_data["Season"].isna()])

,rating_team,EBP_latest_team,NOL_latest_team,POM_latest_team,POM_rolling_team,NOL_rolling_team,EBP_rolling_team,count_reg_team,mean_reg_team,std_reg_team,count_post_team,mean_post_team,std_post_team,TeamName_team,Season_x,FG2Pct_team,FG3Pct_team,FTPct_team,OppFG2Pct_team,OppFG3Pct_team,StlRate_team,OppStlRate_team,FTRate_offense_team,FTRate_defense_team,TOPct_offense_team,ORPct_offense_team,HgtEff_team,Bench_team,AdjTempo_team,AdjOE_team,AdjDE_team,Season_y,rating_opp,EBP_latest_opp,NOL_latest_opp,POM_latest_opp,POM_rolling_opp,NOL_rolling_opp,EBP_rolling_opp,count_reg_opp,mean_reg_opp,std_reg_opp,count_post_opp,mean_post_opp,std_post_opp,TeamName_opp,Season,FG2Pct_opp,FG3Pct_opp,FTPct_opp,OppFG2Pct_opp,OppFG3Pct_opp,StlRate_opp,OppStlRate_opp,FTRate_offense_opp,FTRate_defense_opp,TOPct_offense_opp,ORPct_offense_opp,HgtEff_opp,Bench_opp,AdjTempo_opp,AdjOE_opp,AdjDE_opp,win
0,19.514120,15,6,15,15.0,5.8,15.8,447.0,0.805369,0.396360,42.0,0.690476,0.467901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003,5.517589,64,19,40,38.0,19.4,55.8,460.0,0.623913,0.484930,4.0,0.250000,0.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1,19.514120,15,6,15,15.0,5.8,15.8,447.0,0.805369,0.396360,42.0,0.690476,0.467901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003,13.384155,13,5,13,12.2,6.4,13.0,117.0,0.675214,0.470309,1.0,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
2,18.203812,4,8,4,4.2,8.8,3.2,557.0,0.795332,0.403821,49.0,0.653061,0.480929,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003,2.785359,82,61,99,107.4,65.4,89.2,499.0,0.444890,0.497452,0.0,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
3,18.203812,4,8,4,4.2,8.8,3.2,557.0,0.795332,0.403821,49.0,0.653061,0.480929,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003,13.579332,151,174,136,154.2,200.2,162.8,231.0,0.666667,0.472428,6.0,0.333333,0.516398,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
4,18.203812,4,8,4,4.2,8.8,3.2,557.0,0.795332,0.403821,49.0,0.653061,0.480929,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003,19.514120,15,6,15,15.0,5.8,15.8,447.0,0.805369,0.396360,42.0,0.690476,0.467901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
5,17.363221,20,17,17,22.4,21.4,22.0,359.0,0.721448,0.448912,28.0,0.714286,0.460044,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003,0.530346,361,360,361,360.8,358.4,360.4,136.0,0.404412,0.492592,0.0,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
6,17.363221,20,17,17,22.4,21.4,22.0,359.0,0.721448,0.448912,28.0,0.714286,0.460044,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003,8.180665,62,96,51,44.2,83.0,49.4,449.0,0.743875,0.436978,27.0,0.629630,0.492103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
7,17.363221,20,17,17,22.4,21.4,22.0,359.0,0.721448,0.448912,28.0,0.714286,0.460044,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003,14.307485,21,39,22,16.6,30.0,18.2,112.0,0.589286,0.494175,2.0,0.500000,0.707107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
8,17.363221,20,17,17,22.4,21.4,22.0,359.0,0.721448,0.448912,28.0,0.714286,0.460044,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003,13.384155,13,5,13,12.2,6.4,13.0,117.0,0.675214,0.470309,1.0,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
9,16.452552,216,246,190,183.4,238.0,211.6,357.0,0.764706,0.424778,32.0,0.812500,0.396558,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003,1.966184,194,226,222,224.2,248.2,193.2,352.0,0.494318,0.500679,1.0,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,N